# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [ ]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [ ]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  gr_df_da = (3*(-a**2)) - (0.5*(a**(-0.5)))
  gr_df_db = 3*cos(3*b) + 2.5*(b**1.5)
  gr_df_dc = 1*(c**-2)

  return gr_df_da, gr_df_db, gr_df_dc

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [ ]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

h = 0.000001
f_in = [2, 3, 4]
f_  = f(2, 3, 4)
result= []
for index in range(len(f_in)):
  value = f_in[index]
  new_value = value + h
  f_in[index] = new_value
  f_nv = f(*f_in)
  gr_df_dv = (f_nv - f_)/h
  result.append(gr_df_dv)
  f_in[index] = value
# -----------
numerical_grad = result # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [ ]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.
h = 0.000001
f_in = [2, 3, 4]
f_in_2 = [2, 3, 4]
f_  = f(2, 3, 4)
result= []
for index,index_2 in enumerate(zip(f_in,f_in_2 )):
  value = f_in[index]
  new_value = value + h
  new_value_2 = value - h
  f_in[index] = new_value
  f_in_2[index] = new_value_2
  f_nv = f(*f_in)
  f_n_2 = f(*f_in_2)
  gr_df_dv = (f_nv - f_n_2)/(2*h)
  result.append(gr_df_dv)
  f_in[index] = value
  f_in_2[index] = value

# -----------
numerical_grad2 = result # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027401572
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188


## section 2: support for softmax

In [3]:
# Value class starter code, with many functions taken out
from math import exp, log, log10

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out


    # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  # TODO
  def __sub__(self, other):
     return self + (-other)

  def __neg__(self):
    return self * -1

  def __radd__(self, other): # other + self
    return self + other

  def __mul__(self, other):
    other = other if isinstance(other,Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

  def __pow__(self, other):
    assert isinstance(other, (int, float))
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
      self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out

  def exp(self):
     x = self.data
     out = Value(exp(x), (self, ), 'exp')

     def _backward():
      self.grad += out.data * out.grad

     out._backward = _backward
     return out

  def log(self):
     x = self.data
     out = Value(log10(x), (self,), 'log')

     def _backward():
      self.grad += (self.data ** -1) * out.grad
      print(self.grad)

     out._backward = _backward
     print(f'out._backward  = {out._backward }')
     return out

  def __rmul__(self, other): # other * self
      return self * other

  def __truediv__(self, other): # self / other
    return self * other** -1

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [4]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
print(f'probs is {probs}')
loss = -probs[3].log() # dim 3 acts as the label for this input example
print(f'loss is {loss}')
loss.backward()

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


probs is [Value(data=0.04177257051535045), Value(data=0.839024507462532), Value(data=0.00565330266221633), Value(data=0.11354961935990122)]
out._backward  = <function Value.log.<locals>._backward at 0x7e25292aecb0>
loss is Value(data=0.944814317280191)
-8.806722608469956
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [16]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
a = torch.Tensor([0.0]).double()                ; a.requires_grad = True
b = torch.Tensor([3.0]).double()                ; b.requires_grad = True
c = torch.Tensor([-2.0]).double()               ; c.requires_grad = True
d = torch.Tensor([1.0]).double()                ; d.requires_grad = True
# b = torch.Tensor([6.8813735870195432]).double()  ; b.requires_grad = True
n = softmax([a,b,c,d])

o = -torch.log(n[3])

print(o.data.item())
o.backward()

print('---')
print('a', a.grad.item())
print('b', b.grad.item())
print('c', c.grad.item())
print('d', d.grad.item())

2.1755153626167147
---
a 0.041772570515350445
b 0.8390245074625319
c 0.005653302662216329
d -0.8864503806400988
